$$
p(x)=\int p(z)p(x|z)dz.
$$

$$
\log p(x) \geq \mathbb{E}_{q(z|x)}[\log p(x|z)] - D_{\text{KL}}(q(z|x)||p(z))
$$

- 证据下界 (Evidence Lower Bound, ELBO)
    - 既然无法直接最大化 log p(x)，那我们就去最大化它的下界。下界被抬得越高，log p(x) 的真实值也就会越高。
    - $q(z|x)$: 近似后验分布 (Approximate Posterior) 或 编码器 (Encoder)。
    - $\mathbb{E}_{q(z|x)}[...]$: 期望。意思是“从鉴赏家 q(z|x) 推断出的各种可能的‘想法’中，我们进行采样，然后计算一个平均值”。
- training
    - 第一项: $\mathbb{E}_{q(z|x)}[\log p(x|z)]$ (重构项 Reconstruction Term)
    - 第二项: $D_{\text{KL}}(q(z|x)||p(z))$ (正则化项 Regularization Term)
        - 这一项衡量了“鉴赏家” $q(z|x)$ 推断出的“想法”的分布，与我们预设的“灵感库” $p(z)$ 的分布有多接近。我们希望这个KL散度越小越好（即在它前面有个负号，所以最大化整个ELBO等于最小化KL散度）。
- inference
    - first infer：$q(z|x)$
        - 深度学习训练要做的事情，训练 $q(z|x)$ (编码器) 的全部意义，就是为了训练出一个强大的 p(x|z) (解码器)。
    - generate new data/decoding
        - $z \sim p(z) \xrightarrow{\text{解码器 } p(x|z)} x'$
        - 生成新数据时，我们确实不再使用编码器了。
            - 如果没有编码器在训练阶段的参与，解码器将完全不知道如何将一个随机向量 z 变成一张有意义的图片。

### 变分

- https://www.youtube.com/watch?v=u4BJdBCDR9w

- 用一个简单的、我们已知的概率分布（比如高斯分布 $q(z|x)$），去“近似”一个我们想知道但太复杂、算不出来的真实分布（后验分布 $p(z|x)$)。
    - “变分”的过程，就是寻找最优的 μ 和 σ，让我们定义的这个简单高斯分布 q(z|x)，尽可能地接近那个无法计算的、真实的 p(z|x)。
- 变分贝叶斯
    - 贝叶斯：$p(z|x) = \frac{p(x|z)\cdot p(z)}{p(x)}$
        - $p(x)$: intractable
    - 一个简单的、可控的 $q(z|x)$，然后通过训练（最小化KL散度，等价于最大化ELBO），让 q(z|x) 的参数（由编码器决定）不断调整，直到它成为 p(z|x) 的一个足够好的“高仿品”。